$$
\newcommand{\ped}[1]{_{\mathrm{#1}}}
\newcommand{\ap}[1]{^{\mathrm{#1}}}
\newcommand{\nvector}[1]{\mathbf{#1}}
\newcommand{\nmatrix}[1]{\mathit{#1}}
\newcommand{\unitvector}[1]{\hat{\nvector{e}}_{#1}}
\newcommand{\volume}{\mathcal{V}}
\newcommand{\average}[1]{\overline{#1}}
\newcommand{\rate}[1]{\dot{#1}}
\newcommand{\flux}[1]{{#1}''}
\newcommand{\curl}[1]{\nabla\times {#1}}
\newcommand{\curlv}[1]{\curl{\nvector{#1}}}
\newcommand{\divergent}[1]{\nabla \cdot #1}
\newcommand{\divergentv}[1]{\divergent{\nvector{#1}}}
\newcommand{\divergentpar}[1]{\divergent{\left( #1 \right)}}
\newcommand{\gradient}[1]{\nabla {#1}}
\newcommand{\gradientpar}[1]{\gradient{\left( {#1} \right)}}
\newcommand{\laplacian}[1]{\nabla^2 #1}
\newcommand{\laplacianpar}[1]{\laplacian{\left( #1 \right)}}
\newcommand{\vectornorm}[1]{\left\lVert #1 \right\rVert}
\newcommand{\diffp}[2]{\frac{\partial {#1}}{\partial {#2}}}
\newcommand{\diffps}[2]{\frac{\partial^2 {#1}}{\partial {#2}^2}}
\newcommand{\rvec}{\nvector{r}}
\newcommand{\nvh}{\nvector{H}}
\newcommand{\nvb}{\nvector{B}}
\newcommand{\nvrem}{\nvector{B}\ped{rem}}
\newcommand{\nvbrem}{\nvrem}
\newcommand{\nvm}{\nvector{M}}
\newcommand{\mur}{\mu\ped{r}}
\newcommand{\nvbremhat}{\hat{\nvector{B}}\ped{rem}}
\newcommand{\acoef}[2]{a_{{#1},\mathrm{#2}}}
\newcommand{\bcoef}[2]{b_{{#1},\mathrm{#2}}}
\newcommand{\Azexpr}[1]{A_{\mathrm{#1}\, z}}
\newcommand{\bremii}{B_{\mathrm{rem,II}}}
\newcommand{\bremiv}{B_{\mathrm{rem,IV}}}
\newcommand{\aIII}{\acoef{1}{III}}
\newcommand{\bIII}{\bcoef{1}{III}}
\newcommand{\nvbIII}{\nvector{B}\ped{III}}
\newcommand{\BIII}{B\ped{III}}
\newcommand{\diffd}[1]{\mathrm{d}\,{#1}}
$$

# TeslaMax Optimization

Teslamax model:

<img src="figures/teslamax.png" width=500>

In [1]:
from pathlib import Path
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import teslamax
from teslamax import TeslaMaxModel



mpl_params = {'text.usetex': True,
              'font.family': 'serif',
              'font.serif': 'Computer Modern',
              'text.latex.preamble': [r'\usepackage{engsymbols}',
                                      r'\usepackage{magref}',
                                      r'\usepackage{siunitx}']}
matplotlib.rcParams.update(mpl_params)


In [2]:
os.getcwd()

'C:\\Users\\fabiofortkamp\\code\\TeslaMax'

In [40]:
import importlib
importlib.reload(teslamax)

<module 'teslamax' from 'C:\\Users\\fabiofortkamp\\code\\TeslaMax\\teslamax\\__init__.py'>

## Calculating functional

In [33]:
params_optimization = {"R_i": 0.015,
                "R_o": 0.070,
                "h_gap": 0.020,
                "R_s": 0.140,
                "h_fc": 0.010,
                "R_e": 2,
                "n_IV": 3,
                "phi_S_IV": 45,
                "n_II": 3,
                "phi_C_II": 15,
                "phi_S_II": 45,
                "B_rem_II_1": 1.4,
                "B_rem_II_2": 1.4,
                "B_rem_II_3": 1.4,
                "mu_r_II": 1.05,
                "B_rem_IV_1": 1.4,
                "B_rem_IV_2": 1.4,
                "B_rem_IV_3": 1.4,
                "mu_r_IV": 1.05,
              "linear_iron": 1,
              "mu_r_iron": 5e3,
             }

alpha_B_rem = [15,30,45,15,45,135]

params_optimization = teslamax.expand_parameters_from_remanence_array(alpha_B_rem, params_optimization)

The code below calculate the individual contributions for each magnet block. For instance, the i-th element of the variable `F_II_x` is a list of points in the form $(B_x, B_y)$, calculated in a mesh in the air gap, where only the i-th segment in magnet II has a non-null remanence, with magnitude included in the above dictionary and oriented in the $x$ direction.

In [28]:
delta_alpha = 1e-6

n_II = params_optimization['n_II']
n_IV = params_optimization['n_IV']
    
F_II_x = []
F_II_y = []
    
F_IV_x = []
F_IV_y = []
    
R_o = params_optimization["R_o"]
R_g = params_optimization["R_o"] + params_optimization["h_gap"]
points_air_gap = teslamax.generate_sector_mesh_points(1.001*R_o,0.999*R_g,0.0,np.pi)


for k in range(0,n_II):
    F_II_x.append(teslamax.calculate_B_III_from_single_block(point=points_air_gap, 
                                                   segment=k+1, 
                                                   magnet='II', 
                                                   magnitude=params_optimization["B_rem_II_%d" %(k+1)], 
                                                    angle=0.0, 
                                                    params=params_optimization))
        
        
    F_II_y.append(teslamax.calculate_B_III_from_single_block(point=points_air_gap, 
                                                   segment=k+1,
                                                   magnet='II', 
                                                   magnitude=params_optimization["B_rem_II_%d" %(k+1)], 
                                                    angle=90.0, 
                                                    params=params_optimization))

for j in range(0,n_IV):
    F_IV_x.append(teslamax.calculate_B_III_from_single_block(point=points_air_gap, 
                                                   segment=j+1, 
                                                   magnet='IV', 
                                                   magnitude=params_optimization["B_rem_IV_%d" %(j+1)], 
                                                    angle=0.0, 
                                                    params=params_optimization))
        
        
    F_IV_y.append(teslamax.calculate_B_III_from_single_block(point=points_air_gap, 
                                                   segment=j+1,
                                                   magnet='IV', 
                                                   magnitude=params_optimization["B_rem_IV_%d" %(j+1)], 
                                                    angle=90.0, 
                                                    params=params_optimization))
        

In [32]:
F_II_x[0]

array([[  2.10943446e-02,   2.12489439e-04],
       [  2.08467295e-02,   2.82832666e-04],
       [  2.05941864e-02,   3.94219622e-04],
       ..., 
       [  3.08528880e-02,   2.07535477e-05],
       [  3.07694049e-02,  -2.08181138e-05],
       [  3.06830942e-02,  -7.83296845e-05]])

Next we apply the principle of superposition, combining the contributions calculated above and using the actual remanence magnitude and angle as specified by the dictionary `params_optimization` and by the array `alpha_B_rem`.

In [34]:
B_III = 0

for k in range(0,n_II):
    B_rem = params_optimization["B_rem_II_%d" %(k+1)]
    alpha = alpha_B_rem[k]
    
    B = B_rem * (np.cos(alpha) * F_II_x[k] + np.sin(alpha) * F_II_y[k])
    
    B_III = B_III + B

for j in range(0,n_IV):
    B_rem = params_optimization["B_rem_IV_%d" %(j+1)]
    alpha = alpha_B_rem[n_II + j]
    
    B = B_rem * (np.cos(alpha) * F_II_x[k] + np.sin(alpha) * F_II_y[k])
    
    B_III = B_III + B


In [35]:
B_III

array([[ -4.27497924e-01,   2.38421503e-04],
       [ -4.27776491e-01,  -3.23684756e-03],
       [ -4.28088098e-01,  -6.66906819e-03],
       ..., 
       [ -2.88056271e-01,   7.77604134e-03],
       [ -2.88267108e-01,   3.80167022e-03],
       [ -2.88506612e-01,  -2.11482539e-04]])

To get the full grid information in the form $(x, y, B_x, B_y)$:

This function calculates the functional from a list of such points:

And this function calculates the derivative of the functional in respect to the i-th element of `array_B_rem`. Notice that, due to the Principle of Superposition, the calculation of the derivative requires only one extra FEM simulation.